# Current App 12/31/21

In [1]:
%load_ext autoreload 
%autoreload 2

## NOTE ADDED TO PROJECT FUNCTIONS> data_acquisiton
import project_functions as fn

fsds v0.4.14 loaded.


Package,Handle,Version,Documentation,Imported
pandas,pd,1.1.3,https://pandas.pydata.org/docs/,Y
fsds,fs,0.4.14,https://fs-ds.readthedocs.io/en/latest/,Y
numpy,np,1.21.5,https://numpy.org/doc/stable/reference/,Y
matplotlib,mpl,3.3.1,https://matplotlib.org/stable/api/index.html,Y
matplotlib.pyplot,plt,,https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html#module-matplotlib.pyplot,Y
seaborn,sns,0.11.0,https://seaborn.pydata.org/api.html,Y
IPython.display,dp,,https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html,Y
sklearn,,0.23.2,,N


## Entire App

```python
import streamlit as st

# from fsds.imports import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import os,glob,sys,joblib,zipfile,json
import datetime as dt
import re

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_dark"

plt.rcParams['figure.figsize'] = (12,6)
pd.set_option('display.max_columns',0)
# fs.check_package_versions(['statsmodels'],fpath=True)



### FORECAST SPECIFIC FUNCTIONS
import statsmodels.api as sms
import statsmodels.tsa.api as tsa
from pmdarima import auto_arima
import project_functions as fn
# from fsds import pandemic as fn
import os,json,glob

with open("FILE_DIRECTORY.json") as f:
    FPATHS = json.load(f)


### TItle
st.markdown('# Planning for the Pandemic')
st.markdown("""
- James M. Irving, PhD.
    - james.irving.phd@gmail.com
    - [LinkedIn](https://www.linkedin.com/in/james-irving-4246b571/)
    - [GitHub Repo](https://github.com/jirvingphd/predicting-the-pandemic)

___""")

st.markdown("""## ***Goal***
- Covid-19 and the various strains that have since emerged has upended modern life and fundamentally changed how we function as a society.
- Part of what has made it difficult to tackle the pandemic is the differences between states, state laws/policies, and a lack of public understanding about the predictability of the surges in cases. 
- The goal of this dashboard is to find the provide easy access state-level coronavirus and hospital capacity statistics.
    - Furthermore, I wanted to provide on-demand timeseries forecasts into the near future for all/any of these statistics.
""")



st.markdown('## ***The Data***')
st.markdown('- This dashboard uses data from several APIs and kaggle datasets. To fetch the lateast data, click the button below.')
WORKFLOW_BUTTON = st.button("Fetch new data.",)

st.markdown('> Note: it can take up to 2 minutes to download the data.')

st.markdown("""### Sources
- Coronavirus Data by State- # of Cases/Deaths by State
    - [Kaggle Dataset: "COVID-19 data from John Hopkins University"](https://www.kaggle.com/antgoldbloom/covid19-data-from-john-hopkins-university) 
    - Repackaged version of the data from the [official Johns Hopkins Repository](https://github.com/CSSEGISandData/COVID-19)
- Hospital Hospital & ICU Occupancy Data:
    - [HealthData.gob Api: "COVID-19 Reported Patient Impact and Hospital Capacity by State Timeseries API"](https://healthdata.gov/Hospital/COVID-19-Reported-Patient-Impact-and-Hospital-Capa/g62h-syeh)
""")
# RUN_FULL_WORKFLOW=False


def load_data(WORKFLOW_BUTTON=False):
    if WORKFLOW_BUTTON == True:
        df_states,STATES = fn.data_acquisition.FULL_WORKFLOW(merge_hospital_data=True)
        ## renaming since merge_hofspital_data=True
    #     DF = df_states.copy()
    #     print(STATES.keys())    
        
    else:
        # print(f"[i] Using previously downloaded data...")
        # df_states = pd.read_pickle(FPATHS['fpath_final_df_pickle'])
        df_states =  pd.read_csv(FPATHS['fpath_final_df_csv'],compression='gzip',parse_dates=['Date'],
                    index_col=[0,1])

    #     with open(FPATHS['fpath_final_states']) as f:
        STATES = joblib.load(FPATHS['fpath_final_states'])
    return df_states,STATES

## load data and save options
df, STATES = load_data(WORKFLOW_BUTTON)
options_stats= df.drop(['Deaths','Cases'],axis=1).columns.tolist()

st.markdown("___")
st.markdown("## ***Overview - Comparing All States***")

# calc dates for map
# today = dt.date.today()
# end_state = today
# start_date = pd.Timestamp(today) - pd.Timedelta(f'{str(n_days)} days')
latest_date = df.droplevel(0).index.max()
end_date = latest_date

## plot state map
n_days = st.slider(f"PAST N # OF DAYS BEFORE {latest_date.strftime('%m/%d/%Y')}",value=30,min_value=7,max_value=180)
col = st.selectbox("Which statistic to map?", options_stats)

start_date = pd.Timestamp(latest_date) - pd.Timedelta(f'{str(n_days)} days')


## get map
map = fn.app_functions.plot_map_columns(df,col=col, last_n_days=n_days,
plot_map=False,return_map=True)

# get just df
df_rank= fn.app_functions.plot_map_columns(df,col=col, last_n_days=n_days,
plot_map=False,return_map=False)

# show map
st.plotly_chart(map)

### Plot same stat for different states
st.markdown("___")
st.markdown('## ***Comparing Selected States***')

## select states and stats
stat_to_compare = st.multiselect("Which statistic to compare?",options_stats,
default=["Cases-New"])
states_to_compare = st.multiselect("Which states to compare?",list(STATES.keys()),
default=["NY",'MD','FL','CA','TX'])

## get and show plot
plot_df = fn.app_functions.get_states_to_plot(df,state_list=states_to_compare,
            plot_cols=stat_to_compare,
                            agg_func= 'mean',
                  rename_cols=True,fill_method='interpolate',
                  remove_outliers=False, state_first=True,
                  threshold_type=['0','%'], diagnose=False)
st.plotly_chart(px.line(plot_df))


st.markdown("___")

# ############################## PRIOR TO  09/21 ###########################
st.markdown('## ***Timeseries Forecasting by State/Statistic***')


default_model_start = latest_date - pd.to_timedelta('365 days')
state_name = st.selectbox('Select State', list(STATES.keys()))
col = st.selectbox("Select statistic",options_stats)
start_date = st.date_input('Start Date for Training Data',
 value=default_model_start)#pd.to_datetime('06-2020'))


df_state = STATES[state_name].loc[start_date:].copy()

# # col = 'Cases-New'
ts = df_state[col].copy()
ax = ts.plot(title=f"{state_name}-{col}");
ax.set_ylabel(col)



st.pyplot(ax.get_figure())# plt.show()


st.markdown("""> **Click "`Run model`" below to start the modeling process for the selected state and statistic.**
-  [!] Warning: the gridsearch process may take several minutes. Try selecting a more recent start date to increase performance.""")


model_q = st.button('Run model.', 
on_click= fn.modeling.make_timeseries_model,args=(STATES,state_name,col))



# st.button('Hit me')
# st.checkbox('Check me out')
# st.radio('Radio', [1,2,3])
# st.multiselect('Multiselect', [1,2,3])
# st.slider('Slide me', min_value=0, max_value=10)
# st.select_slider('Slide to select', options=[1,'2'])
# st.text_input('Enter some text')
# st.number_input('Enter a number')
# st.text_area('Area for textual entry')
# st.date_input('Date input')
# st.time_input('Time entry')
# st.file_uploader('File uploader')
# st.color_picker('Pick a color')
```

## Parts of App

In [2]:
# !pip install fsds

```python
import streamlit as st

from fsds.imports import *
import pandas as pd

import os,glob,sys,joblib,zipfile,json
import datetime as dt
import re

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_dark"

plt.rcParams['figure.figsize'] = (12,6)
pd.set_option('display.max_columns',0)
# fs.check_package_versions(['statsmodels'],fpath=True)



### FORECAST SPECIFIC FUNCTIONS
import statsmodels.api as sms
import statsmodels.tsa.api as tsa
from pmdarima import auto_arima
import project_functions as fn
# from fsds import pandemic as fn
import os,json,glob

with open("FILE_DIRECTORY.json") as f:
    FPATHS = json.load(f)
```

```python
### TItle
st.markdown('# Planning for the Pandemic')
st.markdown("""
- James M. Irving, PhD.
    - james.irving.phd@gmail.com
    - [LinkedIn](https://www.linkedin.com/in/james-irving-4246b571/)
    - [GitHub Repo](https://github.com/jirvingphd/predicting-the-pandemic)

___""")

st.markdown("""## ***Goal***
- Covid-19 and the various strains that have since emerged has upended modern life and fundamentally changed how we function as a society.
- Part of what has made it difficult to tackle the pandemic is the differences between states, state laws/policies, and a lack of public understanding about the predictability of the surges in cases. 
- The goal of this dashboard is to find the provide easy access state-level coronavirus and hospital capacity statistics.
    - Furthermore, I wanted to provide on-demand timeseries forecasts into the near future for all/any of these statistics.
""")



st.markdown('## ***The Data***')
st.markdown('- This dashboard uses data from several APIs and kaggle datasets. To fetch the lateast data, click the button below.')
WORKFLOW_BUTTON = st.button("Fetch new data.",)

st.markdown('> Note: it can take up to 2 minutes to download the data.')

st.markdown("""### Sources
- Coronavirus Data by State- # of Cases/Deaths by State
    - [Kaggle Dataset: "COVID-19 data from John Hopkins University"](https://www.kaggle.com/antgoldbloom/covid19-data-from-john-hopkins-university) 
    - Repackaged version of the data from the [official Johns Hopkins Repository](https://github.com/CSSEGISandData/COVID-19)
- Hospital Hospital & ICU Occupancy Data:
    - [HealthData.gob Api: "COVID-19 Reported Patient Impact and Hospital Capacity by State Timeseries API"](https://healthdata.gov/Hospital/COVID-19-Reported-Patient-Impact-and-Hospital-Capa/g62h-syeh)
""")
# RUN_FULL_WORKFLOW=False
```

```python
def load_data(WORKFLOW_BUTTON=False):
    if WORKFLOW_BUTTON == True:
        df_states,STATES = fn.data_acquisition.FULL_WORKFLOW(merge_hospital_data=True)
        ## renaming since merge_hofspital_data=True
    #     DF = df_states.copy()
    #     print(STATES.keys())    
        
    else:
        # print(f"[i] Using previously downloaded data...")
        # df_states = pd.read_pickle(FPATHS['fpath_final_df_pickle'])
        df_states =  pd.read_csv(FPATHS['fpath_final_df_csv'],compression='gzip',parse_dates=['Date'],
                    index_col=[0,1])

    #     with open(FPATHS['fpath_final_states']) as f:
        STATES = joblib.load(FPATHS['fpath_final_states'])
    return df_states,STATES

## load data and save options
df, STATES = load_data(WORKFLOW_BUTTON)
options_stats= df.drop(['Deaths','Cases'],axis=1).columns.tolist()

st.markdown("___")
st.markdown("## ***Overview - Comparing All States***")
## plot state map
n_days = st.slider("PAST N # OF DAYS",value=30,min_value=7,max_value=180)
col = st.selectbox("Which statistic to map?", options_stats)

# calc dates
today = dt.date.today()
end_state = today
start_date = pd.Timestamp(today) - pd.Timedelta(f'{str(n_days)} days')

## get map
map = fn.app_functions.plot_map_columns(df,col=col, last_n_days=n_days,
plot_map=False,return_map=True)

# get just df
df_rank= fn.app_functions.plot_map_columns(df,col=col, last_n_days=n_days,
plot_map=False,return_map=False)

# show map
st.plotly_chart(map)

### Plot same stat for different states
st.markdown("___")
st.markdown('## ***Comparing Selected States***')

## select states and stats
stat_to_compare = st.multiselect("Which statistic to compare?",options_stats,
default=["Cases-New"])
states_to_compare = st.multiselect("Which states to compare?",list(STATES.keys()),
default=["NY",'MD','FL','CA','TX'])

## get and show plot
plot_df = fn.app_functions.get_states_to_plot(df,state_list=states_to_compare,
            plot_cols=stat_to_compare,
                            agg_func= 'mean',
                  rename_cols=True,fill_method='interpolate',
                  remove_outliers=False, state_first=True,
                  threshold_type=['0','%'], diagnose=False)
st.plotly_chart(px.line(plot_df))


st.markdown("___")

# ############################## PRIOR TO  09/21 ###########################
st.markdown('## ***Timeseries Forecasting by State/Statistic***')


default_model_start = today - pd.to_timedelta('365 days')
state_name = st.selectbox('Select State', list(STATES.keys()))
col = st.selectbox("Select statistic",options_stats)
start_date = st.date_input('Start Date for Training Data',
 value=default_model_start)#pd.to_datetime('06-2020'))


df_state = STATES[state_name].loc[start_date:].copy()

# # col = 'Cases-New'
ts = df_state[col].copy()
ax = ts.plot(title=f"{state_name}-{col}");
ax.set_ylabel(col)



st.pyplot(ax.get_figure())# plt.show()


st.markdown("""> **Click "`Run model`" below to start the modeling process for the selected state and statistic.**
-  [!] Warning: the gridsearch process may take several minutes. Try selecting a more recent start date to increase performance.""")


model_q = st.button('Run model.', 
on_click= fn.modeling.make_timeseries_model,args=(STATES,state_name,col))



# st.button('Hit me')
# st.checkbox('Check me out')
# st.radio('Radio', [1,2,3])
# st.multiselect('Multiselect', [1,2,3])
# st.slider('Slide me', min_value=0, max_value=10)
# st.select_slider('Slide to select', options=[1,'2'])
# st.text_input('Enter some text')
# st.number_input('Enter a number')
# st.text_area('Area for textual entry')
# st.date_input('Date input')
# st.time_input('Time entry')
# st.file_uploader('File uploader')
# st.color_picker('Pick a color')

```

# Test Workflow

### Notes on Data Acquisition:

- I should create a function for each dataframe that I create, including:
    - population
    - hospital data
    - kaggle covid data
    
>- **Need to add back normalizing by population**
    - Issue was...? 
        - Old population data, not 2020 census


## Test App Code

In [3]:
import streamlit as st

from fsds.imports import *
import pandas as pd

import os,glob,sys,joblib,zipfile,json
import datetime as dt
import re

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_dark"

plt.rcParams['figure.figsize'] = (12,6)
pd.set_option('display.max_columns',0)
# fs.check_package_versions(['statsmodels'],fpath=True)



### FORECAST SPECIFIC FUNCTIONS
import statsmodels.api as sms
import statsmodels.tsa.api as tsa
from pmdarima import auto_arima
import project_functions as fn
# from fsds import pandemic as fn
import os,json,glob

with open("FILE_DIRECTORY.json") as f:
    FPATHS = json.load(f)
FPATHS

{'fpath_raw': './data_raw/',
 'fpath_clean': './data/',
 'fpath_reference': './reference_data/',
 'fpath_state_data': './data/state_data/',
 'fpath_final_states': './data/FINAL_STATES.joblib',
 'fpath_final_df_csv': './data/FINAL_STATES.csv.gz',
 'fpath_final_df_pickle': './data/FINAL_STATES.pickle',
 'name_of_fpath_vars': ['fpath_raw',
  'fpath_clean',
  'fpath_reference',
  'fpath_state_data',
  'fpath_final_states',
  'fpath_final_df_csv',
  'fpath_final_df_pickle',
  'name_of_fpath_vars']}

In [4]:
def load_data(WORKFLOW_BUTTON=False):
    if WORKFLOW_BUTTON == True:
        df_states,STATES = fn.data_acquisition.FULL_WORKFLOW(merge_hospital_data=True)
        ## renaming since merge_hofspital_data=True
    #     DF = df_states.copy()
    #     print(STATES.keys())    
        
    else:
        # print(f"[i] Using previously downloaded data...")
        # df_states = pd.read_pickle(FPATHS['fpath_final_df_pickle'])
        df_states =  pd.read_csv(FPATHS['fpath_final_df_csv'],compression='gzip',parse_dates=['Date'],
                    index_col=[0,1])

    #     with open(FPATHS['fpath_final_states']) as f:
        STATES = joblib.load(FPATHS['fpath_final_states'])
    return df_states,STATES

In [5]:
WORKFLOW_BUTTON=False
## load data and save options
df, STATES = load_data(WORKFLOW_BUTTON)
options_stats= df.drop(['Deaths','Cases'],axis=1).columns.tolist()

In [6]:
df

Deaths   Cases  ...  Deaths-New  Cases-New
   Date                        ...                       
AK 2020-03-23       0      39  ...         0.0        0.0
   2020-03-24       0      43  ...         0.0        4.0
   2020-03-25       1      50  ...         1.0        7.0
   2020-03-26       1      64  ...         0.0       14.0
   2020-03-27       1      75  ...         0.0       11.0
...               ...     ...  ...         ...        ...
WY 2021-12-26    1526  114242  ...         0.0        0.0
   2021-12-27    1526  114624  ...         0.0      382.0
   2021-12-28    1526  114917  ...         0.0      293.0
   2021-12-29    1526  115242  ...         0.0      325.0
   2021-12-30    1526  115638  ...         0.0      396.0

[34943 rows x 6 columns]

In [7]:
df.loc['MD']

,Deaths,Cases,Hospitalized Currently,ICU-Covid Currently,Deaths-New,Cases-New
Date,,,,,,
2020-03-01,0,0,0.0,0.0,0.0,0.0
2020-03-02,0,0,0.0,0.0,0.0,0.0
2020-03-03,0,0,0.0,0.0,0.0,0.0
2020-03-04,0,0,0.0,0.0,0.0,0.0
2020-03-05,0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
2021-12-26,11255,663414,1654.0,340.0,0.0,25035.0
2021-12-27,11255,668790,1805.0,367.0,0.0,5376.0
2021-12-28,11672,675364,1929.0,375.0,417.0,6574.0


In [8]:
## 
state_df = df.loc['MD'].copy()
state_df

,Deaths,Cases,Hospitalized Currently,ICU-Covid Currently,Deaths-New,Cases-New
Date,,,,,,
2020-03-01,0,0,0.0,0.0,0.0,0.0
2020-03-02,0,0,0.0,0.0,0.0,0.0
2020-03-03,0,0,0.0,0.0,0.0,0.0
2020-03-04,0,0,0.0,0.0,0.0,0.0
2020-03-05,0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
2021-12-26,11255,663414,1654.0,340.0,0.0,25035.0
2021-12-27,11255,668790,1805.0,367.0,0.0,5376.0
2021-12-28,11672,675364,1929.0,375.0,417.0,6574.0


In [9]:
state_df.index

DatetimeIndex(['2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04',
               '2020-03-05', '2020-03-06', '2020-03-06', '2020-03-07',
               '2020-03-08', '2020-03-09',
               ...
               '2021-12-21', '2021-12-22', '2021-12-23', '2021-12-24',
               '2021-12-25', '2021-12-26', '2021-12-27', '2021-12-28',
               '2021-12-29', '2021-12-30'],
              dtype='datetime64[ns]', name='Date', length=672, freq=None)

In [10]:
# fn.data_acquisition.

In [11]:
# map_ = fn.app_functions.plot_map_columns(state_df,col="Cases-New", last_n_days=30,
# plot_map=False,return_map=True)

In [12]:
# ##### MAP CODE
# # calc dates for map
# # today = dt.date.today()
# # end_state = today
# # start_date = pd.Timestamp(today) - pd.Timedelta(f'{str(n_days)} days')
# latest_date = df.droplevel(0).index.max()
# end_date = latest_date

# ## plot state map
# n_days = st.slider(f"PAST N # OF DAYS BEFORE {latest_date.strftime('%m/%d/%Y')}",value=30,min_value=7,max_value=180)
# col = st.selectbox("Which statistic to map?", options_stats)

# start_date = pd.Timestamp(latest_date) - pd.Timedelta(f'{str(n_days)} days')


# ## get map
# map_ = fn.app_functions.plot_map_columns(df,col=col, last_n_days=n_days,
# plot_map=False,return_map=True)

# # get just df
# df_rank= fn.app_functions.plot_map_columns(df,col=col, last_n_days=n_days,
# plot_map=False,return_map=False)

# # show map
# st.plotly_chart(map)


In [13]:
# STATES.keys()

In [14]:
df

Deaths   Cases  ...  Deaths-New  Cases-New
   Date                        ...                       
AK 2020-03-23       0      39  ...         0.0        0.0
   2020-03-24       0      43  ...         0.0        4.0
   2020-03-25       1      50  ...         1.0        7.0
   2020-03-26       1      64  ...         0.0       14.0
   2020-03-27       1      75  ...         0.0       11.0
...               ...     ...  ...         ...        ...
WY 2021-12-26    1526  114242  ...         0.0        0.0
   2021-12-27    1526  114624  ...         0.0      382.0
   2021-12-28    1526  114917  ...         0.0      293.0
   2021-12-29    1526  115242  ...         0.0      325.0
   2021-12-30    1526  115638  ...         0.0      396.0

[34943 rows x 6 columns]

### NOTES:

- [x] Will want to change how the start date is determined. Instead of creating `today`, get the last date from one of the states

```python
## CURRENT WAY
# calc dates
today = dt.date.today()
end_state = today
start_date = pd.Timestamp(today) - pd.Timedelta(f'{str(n_days)} days')


## PROPOSED NEW WAY
latest_date = df.droplevel(0).index.max()
end_date = latest_date
start_date = pd.Timestamp(latest_date) - pd.Timedelta(f'{str(n_days)} days')
```

## Updating Functions

In [15]:
## from project functions > app finctions
def plot_map_columns(DF,col='Cases-New',last_n_days=90,
               map_metric='sum',plot_map=True,return_map=False):
    """
    """
    import datetime as dt

    today = dt.date.today()
    end_date = today
    start_date = pd.Timestamp(today) - pd.Timedelta(f'{last_n_days} days')

    plot_df = get_states_to_plot(DF,state_list=None, plot_cols=col,
                             agg_func= 'mean',
                      rename_cols=True,fill_method='interpolate',
                      remove_outliers=True, state_first=True,
                      threshold_type=['0','%'], diagnose=False)

    plot_df.columns = [c.split(' - ')[0] for c in plot_df.columns]
    plot_df = plot_df.loc[start_date:end_date]
    
    
    
    agg_data = plot_df.agg(map_metric).reset_index()
    agg_data.columns= ['state',col]
    
    color_column = col
    map_ = px.choropleth(agg_data,color=color_column,locations='state',
                    locationmode="USA-states", scope='usa', 
                            title=f"{map_metric.title()} {col} by State (Last {last_n_days} Days)",#.format(color_column),
                    color_continuous_scale=px.colors.sequential.Reds)
    if plot_map:
        map_.show(config={'scrollZoom':False})

    if return_map:
            return map_
    else:
        return agg_data
    
    
    
def get_states_to_plot(DF,state_list=["NY",'MD','TX','PA', 'FL'],
                       plot_cols=None, 
                      agg_func= 'mean',
              rename_cols=True,fill_method='interpolate',
              remove_outliers=False, state_first=False,
              threshold_type=['0','%'], diagnose=False):
    
    get_states_kwargs = dict(rename_cols=rename_cols,
                          fill_method=fill_method,
                          state_first=state_first,
                           threshold_type=threshold_type,
                          diagnose=diagnose)
    
    if state_list is None:   
        if isinstance(DF,pd.DataFrame):
            state_list = list(DF.index.get_level_values(0).unique())
    
        elif isinstance(DF,dict):
            state_list = list(sorted(DF.keys()))
    
    ## Get each state
    dfs_to_concat = []
    for state in state_list:
        dfs = get_state_df_ts_final(DF,state,ts_col=plot_cols,**get_states_kwargs)
        dfs_to_concat.append(dfs)
        
        
     ## Concatenate final dfs
    try:
        plot_df = pd.concat(dfs_to_concat,axis=1)#[STATES[s] for s in plot_states],axis=1).iplot()
        new_order = sorted(plot_df.columns.to_list())
        plot_df = plot_df[new_order]
    except:
        print('[!] pd.concat failed, returning list...')
        plot_df = dfs_to_concat
    return plot_df



############################################################### PRE-09/21/21
def get_state_df_ts(STATES, state_name,ts_col=None,group_col='state',# legacy compatible
                     freq='D', start_date='08-2020', agg_func='mean', #formerly sum
                    fill_nans=True, fill_method='interpolate',
                    rename_cols=True,name_sep=' - ',state_first=True,diagnose=True):
    """Take df_us and extracts state's data as then Freq/Aggregation provided
    
    
    Paramters:
    
        - DF
        - state_name
        - ts_col
        - group_col
        - freq 
        - start_date
        - agg_func
        -  fill_nans
        - fill_method
    
    """
    import inspect
    args = inspect.signature(get_state_df_ts)
    args = {k:v.default for k,v in args.parameters.items()}  
    
    
    if isinstance(STATES,dict):
        state_df = STATES[state_name].copy()
        
    elif isinstance(STATES, pd.DataFrame):
        ## Slicing out state df for index dates 
        state_df = STATES.loc[state_name].copy()

    else:
        ## Get state_df group
        state_df = STATES.groupby(group_col).get_group(state_name)#.resample(freq).agg(agg)

        
    ## visualize pre-resampling
    if diagnose:
        pfig = px.line(state_df,title='Pre-Resampling')
        pfig.show()
    
    
    if agg_func=='as_freq':
        try:
            state_df = state_df.resample(freq).asfreq(freq)
        except Exception as e:

            agg_func = args['agg_func']
            state_df = state_df.resample(freq).agg(agg_func)
            print(f"[!] Erorr using agg_func='as_freq'; Using default agg_func ('{agg_func}') instead.")
            print('\tError message below:')
            print("\t",e)        
    elif agg_func is None:
        pass
    else:
        ## Resample and aggregate state data
        state_df = state_df.resample(freq).agg(agg_func)
        
    ## Slice out time period desired.
    state_df = state_df.loc[start_date:]
    

    
    
    ## Deal with null values:
    if fill_method == 'interpolate':
        state_df = state_df.interpolate()
        
    elif fill_method == None:
        pass
    
    else:
        state_df = state_df.fillna(method=fill_method)
        
        
        

     ## Renamed columns with state name
    if rename_cols == True:
    
        ## Get and Rename Sum Cols 
        orig_cols = state_df.columns

        for col in orig_cols:

            if state_first==True:
                new_col_name = f"{state_name}{name_sep}{col}"
            else:
                new_col_name = f"{col}{name_sep}{state_name}"

            state_df[new_col_name] = state_df[col].copy()

        ## Drop original cols
        state_df.drop(orig_cols,axis=1,inplace=True)
    
    
    
    ## Return only columns containing ts_cols
    if ts_col is not None:

        if isinstance(ts_col,str):
            ts_col = [ts_col]

            # find cols that end with the column name
        selected_cols=[]
        for col in ts_col:
            selected_cols.extend([c for c in state_df.columns if c.endswith(col)])
            
        state_df = state_df[selected_cols]
        
    ## Add outlier removal here:
        
        
        
    ## Visualize post-resampling 
    if diagnose:
            pfig = px.line(state_df,title="post-Resampling")
            pfig.show()
    
    
    return state_df





### BEST VERSION YET! 09/12 ######
def get_state_df_ts_final(STATES, state_name,ts_col=None,group_col='state',# legacy compatible
                     freq='D', start_date='08-2020', agg_func='mean', #formerly sum
                    fill_nans=True, fill_method='interpolate',
                    rename_cols=True,name_sep=' - ',state_first=False,
                    remove_outliers=True,  n_diff=1, threshold_type='pct_change',
                      raw_thresh=100, pct_thresh=50, 
                          diagnose=True):
    """Take df_us and extracts state's data as then Freq/Aggregation provided
    
    
    Paramters:
    
        - DF
        - state_name
        - ts_col
        - group_col
        - freq 
        - start_date
        - agg_func
        -  fill_nans
        - fill_method
        
        
    Note on order:
    1. make state_df
    2. resample and aggregate
    3. slice start_date
    4. Select columns
    5. Remove Outliers
    6. Fill Null values
    7. Rename columns
    
    """
    ## Get default arguments for try/except
    import inspect
    args = inspect.signature(get_state_df_ts)
    args = {k:v.default for k,v in args.parameters.items()}  
    
    
    ## Slice state_df depending on which datatype 
    if isinstance(STATES,dict):
        state_df = STATES[state_name].copy()
        
    elif isinstance(STATES, pd.DataFrame):
        ## Slicing out state df for index dates 
        state_df = STATES.loc[state_name].copy()

    else:
        ## Get state_df group
        state_df = STATES.groupby(group_col).get_group(state_name)#.resample(freq).agg(agg)

        
        
    ## visualize pre-resampling
    if diagnose:
        pfig = px.line(state_df,title='Pre-Resampling')
        pfig.show()
    

    ## Resampling and Aggregating 
    if agg_func=='as_freq':
        try:
            state_df = state_df.resample(freq).asfreq(freq)
        except Exception as e:

            agg_func = args['agg_func']
            state_df = state_df.resample(freq).agg(agg_func)
            print(f"[!] Erorr using agg_func='as_freq'; Using default agg_func ('{agg_func}') instead.")
            print('\tError message below:')
            print("\t",e)        
            
    elif agg_func is None:
        pass
    
    else:
        state_df = state_df.resample(freq).agg(agg_func)
        
        
    ## Slice out time period desired.
    state_df = state_df.loc[start_date:]
    
    
    ## Return only columns containing ts_cols
    if ts_col is not None:

        if isinstance(ts_col,str):
            ts_col = [ts_col]

        # find cols that end with the column name
        selected_cols=[]
        for col in ts_col:
            selected_cols.extend([c for c in state_df.columns if c.endswith(col)])
            
        state_df = state_df[selected_cols]
        
        
    ## Remove Outleirs
    if remove_outliers:
        
        if isinstance(threshold_type,str):
            threshold_type= [threshold_type]
            
        for thresh_type in threshold_type:
            state_df = remove_outliers_ts(state_df,threshold_type=thresh_type,
                                         raw_thresh=raw_thresh,pct_thresh=pct_thresh,
                                         n_diff=n_diff,fill_method=fill_method)
        
   
    
    ## Deal with reamaining  null values:  (REMOVE??)
    if fill_method == 'interpolate':
        state_df = state_df.interpolate()
        
    elif fill_method == None:
        pass
    
    else:
        state_df = state_df.fillna(method=fill_method)
    
        

     ## Rename columns with state name
    if rename_cols == True:
    
        ## Get and Rename Sum Cols 
        orig_cols = state_df.columns

        for col in orig_cols:

            if state_first==True:
                new_col_name = f"{state_name}{name_sep}{col}"
            else:
                new_col_name = f"{col}{name_sep}{state_name}"

            state_df[new_col_name] = state_df[col].copy()

        ## Drop original cols
        state_df.drop(orig_cols,axis=1,inplace=True)
    

    ## Visualize post-resampling 
    if diagnose:
            pfig = px.line(state_df,title="post-Resampling")
            pfig.show()

    
    
    return state_df





def calc_perc_change(ts,periods=1,replace_inf_with_nan=True):
    """Calculated the pct_change with pandas and optionall replaces inf values"""
    ts_pct_change = ts.pct_change(periods=periods)#
    
    if replace_inf_with_nan:
        ts_pct_change = ts_pct_change.replace({np.inf:np.nan,
                                      -np.inf:np.nan})

    return ts_pct_change


def remove_outliers_ts(ts_,threshold_type='pct_change',raw_thresh=100,
                       pct_thresh=50, n_diff=1,fill_method='interpolate'):
    """ Remove outliers from time series.
    
    Parameters:
        - ts 
        - threshold_type {'raw',('pct_change','%'),('zero','0')}
        - raw_thresh 
        - pct_thresh
        - n_diff (1) - period for .diff or .pct_change
    """
    
    if isinstance(ts_, pd.Series):
        ts_df = ts_.to_frame(ts_.name)
    else:
        ts_df = ts_.copy()
    
    ## sve copy to remove outleirs from
    ts_out = ts_df.copy()
    
    for col in ts_df.columns:
        ts = ts_df[col].copy()
        
        ## use threshold techniques to identify outleirs
        if threshold_type == 'raw':
            ## saving deltas 
            deltas  = ts.diff(n_diff)
            idx_outliers = deltas.abs()>raw_thresh

        elif (threshold_type == '%') | (threshold_type == 'pct_change'):
            deltas = calc_perc_change(ts,periods=n_diff)
            idx_outliers = deltas.abs() > pct_thresh


        elif (threshold_type == 'zero') | (threshold_type == '0'): 
            deltas = ts.copy()
            idx_outliers = ts == 0
        else:
            raise Exception("Other threshold_kinds are not yet implemented.")


        ## SSaving outleirs
        outliers = deltas[idx_outliers]


        ## Filling in outliers
#         ts_out = ts.copy()
        ts_out[col].loc[outliers.index] = np.nan
    
    
    

        ## Deal with null values:
        if fill_method == 'interpolate':
            ts_out[col] = ts_out[col].interpolate()

        elif fill_method == None:
            pass

        else:
            ts_out[col] = ts_out[col].fillna(method=fill_method)

    
    return ts_out
     
    


In [16]:
map_ = plot_map_columns(df, plot_map=False, return_map=True)
map_

In [17]:
## get and show plot
states_to_compare=['FL','MD','SC']
stat_to_compare = 'Cases-New'
plot_df = fn.app_functions.get_states_to_plot(df,state_list=states_to_compare,
            plot_cols=stat_to_compare,
                            agg_func= 'mean',
                  rename_cols=True,fill_method='interpolate',
                  remove_outliers=False, state_first=True,
                  threshold_type=['0','%'], diagnose=False)
plot_df

,FL - Cases-New,MD - Cases-New,SC - Cases-New
Date,,,
2020-08-01,9642.000000,1019.0,1583.000000
2020-08-02,7104.000000,909.0,1189.000000
2020-08-03,4752.000000,870.0,1163.000000
2020-08-04,5446.000000,710.0,1239.000000
2020-08-05,5409.000000,572.0,1282.000000
...,...,...,...
2021-12-26,44020.666667,25035.0,8918.000000
2021-12-27,28422.000000,5376.0,11152.333333
2021-12-28,29059.000000,6574.0,13386.666667


In [18]:
plot_df_W = plot_df.resample("W").sum()
plot_df_W

,FL - Cases-New,MD - Cases-New,SC - Cases-New
Date,,,
2020-08-02,16746.000000,1928.0,2772.0
2020-08-09,45674.000000,5229.0,8647.0
2020-08-16,40610.000000,4709.0,6062.0
2020-08-23,27155.000000,3890.0,5491.0
2020-08-30,21015.000000,3689.0,6336.0
...,...,...,...
2021-12-05,70413.532468,9508.0,13512.5
2021-12-12,94368.142857,9087.0,11912.0
2021-12-19,171996.342857,17381.0,12385.0


In [19]:
# pfig = px.line(plot_df_W,y="MD - Cases-New",animation_frame="Date")

### Notes re: lineplot

- [x] Would like to have a call back function (or modify `fn.app_functions.get_states_to_plot`) to optionally plot a rolling mean.
    - Added a checkbox
    
- Need to add back normalizing by population

In [20]:
## Apply smooothing
px.line(plot_df.rolling(7).mean())

## Adding Population

In [21]:
# from fn.legacy_data_acquisition
def download_world_pop(data_folder = "./reference_data/",load=True):
    """Downloads world pop zip from kaggle"""

    # Download kaggle dataset
    os.system(f'kaggle datasets download -p {data_folder} -d tanuprabhu/population-by-country-2020')

    ## Specify file and target folder
    file = 'population-by-country-2020.zip'
    target = os.path.join(data_folder,file)

    ## Move zip file to target
#     shutil.move(file,target)
    print(f'Population data saved to {target}')
    
    ## Load csv 
    if load:
        df = pd.read_csv(target)
    else:
        df = target
    return df

In [22]:
##
pop_df = download_world_pop()
pop_df

Population data saved to ./reference_data/population-by-country-2020.zip


,Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,China,1440297825,0.39 %,5540090,153,9388211,-348399.0,1.7,38,61 %,18.47 %
1,India,1382345085,0.99 %,13586631,464,2973190,-532687.0,2.2,28,35 %,17.70 %
2,United States,331341050,0.59 %,1937734,36,9147420,954806.0,1.8,38,83 %,4.25 %
3,Indonesia,274021604,1.07 %,2898047,151,1811570,-98955.0,2.3,30,56 %,3.51 %
4,Pakistan,221612785,2.00 %,4327022,287,770880,-233379.0,3.6,23,35 %,2.83 %
...,...,...,...,...,...,...,...,...,...,...,...
230,Montserrat,4993,0.06 %,3,50,100,NaN,N.A.,N.A.,10 %,0.00 %
231,Falkland Islands,3497,3.05 %,103,0,12170,NaN,N.A.,N.A.,66 %,0.00 %
232,Niue,1628,0.68 %,11,6,260,NaN,N.A.,N.A.,46 %,0.00 %
233,Tokelau,1360,1.27 %,17,136,10,NaN,N.A.,N.A.,0 %,0.00 %


### Potential US Pop Sources

- https://www.kaggle.com/zusmani/us-census-2020
    - `kaggle datasets download -d zusmani/us-census-2020`
    - Cons: 12 GB big!
      
- https://www.census.gov/data/datasets/2020/dec/2020-census-redistricting-summary-file-dataset.html

>- HOW TO!! https://towardsdatascience.com/accessing-census-data-with-python-3e2f2b56e20d

In [23]:
# folder = "/Users/codingdojo//Downloads/"
# files_list = sorted(glob.glob(folder+'**/*.sas',recursive=True))
# files_list

In [24]:
# pd.read_sas(files_list[2],)

In [25]:
# pd.__version__

### `censusdata`
- Source: https://towardsdatascience.com/accessing-census-data-with-python-3e2f2b56e20d

In [26]:
# !pip install CensusData

In [27]:
# import censusdata
# censusdata.download?

In [28]:
# res = censusdata.download('acs3',2020)

In [29]:
# censusdata.censusgeo('')

### `census`


> https://github.com/datamade/census

In [30]:
!pip install census
!pip install us

In [31]:
import json
with open('/Users/codingdojo/.secret/census.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

- First, get yourself a Census API key: https://api.census.gov/data/key_signup.html

```python
from census import Census
from us import states

c = Census("MY_API_KEY")
c.acs5.get(('NAME', 'B25034_010E'),
          {'for': 'state:{}'.format(states.MD.fips)})
```


In [32]:
from census import Census
from us import states

c = Census(login['api-key'])
c.acs5.get(('NAME', 'B25034_010E'),
          {'for': 'state:{}'.format(states.MD.fips)})

[{'NAME': 'Maryland', 'B25034_010E': 128476.0, 'state': '24'}]

In [33]:
# c.acs5.state("POP",states.MD.fips)

In [34]:
# import requests
# s = requests.session()
# s.headers.update({'User-Agent': 'census-demo/0.0'})

# c = Census("MY_API_KEY", session=s)

### ⭐️Option 3!


- https://www.census.gov/data/developers/data-sets/popest-popproj.html
    - https://www.census.gov/data/developers/data-sets/popest-popproj/popest.html
    - Example: https://api.census.gov/data/2021/pep/population/examples.html

In [35]:
import json, requests,os
# with open('/Users/codingdojo/.secret/census.json') as f:
#     login = json.load(f)
# login.keys()
login={'api-key': os.environ['CENSUS_API_KEY']}
login.keys()

dict_keys(['api-key'])

In [36]:
## ALSO ADDED TO BASH_PROFILE AS CENSUS_API_KEY
# os.environ['CENSUS_API_KEY']

In [37]:
# url = f"https://api.census.gov/data/2021/pep/population?get=DENSITY_2021,POP_2021,NAME,STATE&for=region:*&key={login['api-key']}"

# Ex from https://api.census.gov/data/2021/pep/population/examples.html
url_ex = f"https://api.census.gov/data/2021/pep/population?get=NAME,POP_2021&for=state:*&key={login['api-key']}"

url_ex

'https://api.census.gov/data/2021/pep/population?get=NAME,POP_2021&for=state:*&key=3893033f682428f0e5a14542f9c15dae8359a3ff'

In [38]:
resp = requests.get(url_ex)
resp.status_code

200

In [39]:
res_json = resp.json()
res_json

[['NAME', 'POP_2021', 'state'],
 ['Oklahoma', '3986639', '40'],
 ['Nebraska', '1963692', '31'],
 ['Hawaii', '1441553', '15'],
 ['South Dakota', '895376', '46'],
 ['Tennessee', '6975218', '47'],
 ['Nevada', '3143991', '32'],
 ['New Mexico', '2115877', '35'],
 ['Iowa', '3193079', '19'],
 ['Kansas', '2934582', '20'],
 ['District of Columbia', '670050', '11'],
 ['Texas', '29527941', '48'],
 ['Missouri', '6168187', '29'],
 ['Arkansas', '3025891', '05'],
 ['Michigan', '10050811', '26'],
 ['New Hampshire', '1388992', '33'],
 ['North Carolina', '10551162', '37'],
 ['Ohio', '11780017', '39'],
 ['South Carolina', '5190705', '45'],
 ['Wyoming', '578803', '56'],
 ['California', '39237836', '06'],
 ['North Dakota', '774948', '38'],
 ['Louisiana', '4624047', '22'],
 ['Maryland', '6165129', '24'],
 ['Delaware', '1003384', '10'],
 ['Pennsylvania', '12964056', '42'],
 ['Georgia', '10799566', '13'],
 ['Oregon', '4246155', '41'],
 ['Minnesota', '5707390', '27'],
 ['Colorado', '5812069', '08'],
 ['New Jer

In [40]:
pop_df = pd.DataFrame(res_json[1:],columns=res_json[0])

In [41]:
import us
state = us.states.lookup('maryland')
state.fips


'24'

In [42]:
import json, requests,us
# with open('/Users/codingdojo/.secret/census.json') as f:
#     login = json.load(f)
# login.keys()

def get_state_pop_data(use_env_var=True, api_cred_file = '/Users/codingdojo/.secret/census.json'):
    
    if use_env_var:
        login = {'api-key': os.environ['CENSUS_API_KEY']}

    else:
        with open(api_cred_file) as f:
            login = json.load(f)
            
            
    url_ex = f"https://api.census.gov/data/2021/pep/population?get=NAME,POP_2021&for=state:*&key={login['api-key']}"
    resp = requests.get(url_ex)

    ## Convert result to dataframe
    res_json = resp.json()
    pop_df = pd.DataFrame(res_json[1:],columns=res_json[0])
    
    ## Add state abbrev
    pop_df["abbr"] = pop_df['NAME'].map(lambda x: us.states.lookup(x.lower()).abbr)
    pop_df.rename({'state':'fips'},axis=1, inplace=True)
    pop_df = pop_df.sort_values('NAME')
    pop_df = pop_df.reset_index(drop=True)
    
    pop_df['POP_2021'] = pop_df['POP_2021'].astype(int)
    pop_df['fips'] = pop_df['fips'].astype(int)
    return pop_df


In [43]:
df_pop = get_state_pop_data()
df_pop.sort_values('NAME')

,NAME,POP_2021,fips,abbr
0,Alabama,5039877,1,AL
1,Alaska,732673,2,AK
2,Arizona,7276316,4,AZ
3,Arkansas,3025891,5,AR
4,California,39237836,6,CA
5,Colorado,5812069,8,CO
6,Connecticut,3605597,9,CT
7,Delaware,1003384,10,DE
8,District of Columbia,670050,11,DC
9,Florida,21781128,12,FL


In [44]:
%load_ext autoreload 
%autoreload 2

## NOTE ADDED TO PROJECT FUNCTIONS> data_acquisiton
import project_functions as fn
fn.data_acquisition.get_state_pop_data()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,NAME,POP_2021,fips,abbr
0,Alabama,5039877,1,AL
1,Alaska,732673,2,AK
2,Arizona,7276316,4,AZ
3,Arkansas,3025891,5,AR
4,California,39237836,6,CA
5,Colorado,5812069,8,CO
6,Connecticut,3605597,9,CT
7,Delaware,1003384,10,DE
8,District of Columbia,670050,11,DC
9,Florida,21781128,12,FL


## Calc per capita on the fly

In [45]:
## get and show plot
states_to_compare=['FL','MD','SC']
stat_to_compare = 'Cases-New'
plot_df = fn.app_functions.get_states_to_plot(df,state_list=states_to_compare,
            plot_cols=stat_to_compare,
                            agg_func= 'mean',
                  rename_cols=True,fill_method='interpolate',
                  remove_outliers=False, state_first=True,
                  threshold_type=['0','%'], diagnose=False)
plot_df

,FL - Cases-New,MD - Cases-New,SC - Cases-New
Date,,,
2020-08-01,9642.000000,1019.0,1583.000000
2020-08-02,7104.000000,909.0,1189.000000
2020-08-03,4752.000000,870.0,1163.000000
2020-08-04,5446.000000,710.0,1239.000000
2020-08-05,5409.000000,572.0,1282.000000
...,...,...,...
2021-12-26,44020.666667,25035.0,8918.000000
2021-12-27,28422.000000,5376.0,11152.333333
2021-12-28,29059.000000,6574.0,13386.666667


In [46]:
## Download pop data
pop_df = fn.data_acquisition.get_state_pop_data()#get_state_pop_data()
pop_df = pop_df.set_index('abbr')

## Find state names in the column headers
# BUT! already have states_to_compare list
states_to_compare

## get the corresponding  population estimate
for state in states_to_compare:
    state_cols = [c for c in plot_df.columns if state in c]
    
    for col in state_cols:
        pop_adj = (plot_df[col] / pop_df.loc[state, "POP_2021"]) * 100_000
        
        plot_df[f"{col} (per 100K)"] = pop_adj

## divide col / populaton *100_000

In [47]:
plot_df

,FL - Cases-New,MD - Cases-New,SC - Cases-New,FL - Cases-New (per 100K),MD - Cases-New (per 100K),SC - Cases-New (per 100K)
Date,,,,,,
2020-08-01,9642.000000,1019.0,1583.000000,44.267680,16.528446,30.496821
2020-08-02,7104.000000,909.0,1189.000000,32.615391,14.744217,22.906330
2020-08-03,4752.000000,870.0,1163.000000,21.817052,14.111627,22.405434
2020-08-04,5446.000000,710.0,1239.000000,25.003296,11.516385,23.869590
2020-08-05,5409.000000,572.0,1282.000000,24.833425,9.277989,24.697994
...,...,...,...,...,...,...
2021-12-26,44020.666667,25035.0,8918.000000,202.104623,406.074228,171.807105
2021-12-27,28422.000000,5376.0,11152.333333,130.489110,87.200122,214.851997
2021-12-28,29059.000000,6574.0,13386.666667,133.413660,106.631994,257.896888


In [48]:
plot_df_smooth = plot_df.rolling(7).mean()

px.line(plot_df_smooth[[c for c in plot_df.columns if 'per 100' in c]])

In [49]:

px.line(plot_df_smooth.drop(columns=([c for c in plot_df.columns if 'per 100' in c])))